In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from denn import *
import pandas as pd
import io #Used as buffer

In [2]:
path= Path(f'../../data/cluster_results')

In [3]:
def read_csv(f):
    df = pd.read_csv(f).mean()
    df['experiment'] = f.parent.parent.parent.name
    df['function'] = f.parent.parent.name
    name = f.parent.name
    #print(name)
    df['freq'] = int(name[4:].split('n')[0])
    #print(df['freq'])
    df['method'] = f.name.split('_')[0]
    if df['freq']==500:
        df['ns']=int(name[21:].split('n')[0])
        df['tw']=int(name[27:].split('n')[0])
    else:
        df['ns']=int(name[22:].split('n')[0])
        df['tw']=int(name[28:].split('n')[0])  
    if len(name)>29:
        if df['freq']==500:
            df['bs']=int(name[33:].split('n')[0])
            df['epochs']=int(name[42:][0]) 
        else:
            df['bs']=int(name[34:].split('n')[0])
            df['epochs']=int(name[43:][0])
    return df

measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=4

In [5]:
m = measure_name[index_measure]

files = list(path.glob(f'exp*/*/*nn_s2nn_tw5*bs4*/*{m}.csv'))
#print(files)
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=['experiment','function','freq','ns','tw', 'bs','epochs' ],columns='method' ) #,'freq' later for adding frequency experiment, ,values=m 'ns','tw', 'bs','epochs'
data.style.format("{:,.2f}")

ValueError: invalid literal for int() with base 10: '_tw5'

In [7]:
%debug

> <ipython-input-3-997b4713c04f>(14)read_csv()
     12         df['tw']=int(name[27:].split('n')[0])
     13     else:
---> 14         df['ns']=int(name[22:].split('n')[0])
     15         df['tw']=int(name[28:].split('n')[0])
     16     if len(name)>29:

ipdb> name[20:]
'nn_tw5nn_bs4nn_epoch3'
ipdb> q


In [17]:
f'freq{0.5:.1f}'

'freq0.5'

In [ ]:
m = measure_name[0]

files = list(path.glob(f'exp*/*/*nn_s2nn_tw5*bs4*/*{m}.csv'))
df = pd.DataFrame([read_csv(f) for f in files])
# data = pd.pivot_table(df, index=['experiment','function'], columns='method', values=m) #,'freq' later for adding frequency experiment
data = pd.pivot_table(df, index=['experiment','function','freq', 'ns', 'tw','bs','epochs'],columns='method' ) #,'freq' later for adding frequency experiment, ,values=m
data.style.format("{:,.2f}")

In [ ]:
print(data.to_csv(float_format='%.2f'))

In [84]:
!pwd

/Users/maryam/DENN/nbs/maryam


In [100]:
data.to_csv('tmp.csv', float_format='%.2f')

In [102]:
!rm tmp.csv

In [51]:
Func_names=['sphere','rastrigin','rosenbrock']
exp_names=['exp1','exp2','exp3','exp4']
frequency=1000
nn_window=5


In [52]:
#Experiment = Enum('Experiment', 'exp1 exp2 exp3 exp4')
#Method = Enum('Methods', 'noNNRestart noNNReval NNnorm NNdrop')
#Mechname:Random, Worst, Closest
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=0
total=[]
#lables = ['noNNReval', 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']
lables = ['noNNReval', 'NNnorm_Worst', 'NNdrop_Worst']

for experiment in exp_names:
    
    for func in Func_names:
       # path = Path(f'../../data/results/{experiment}/{func}')
        path_in = Path(f'../../data/cluster_results/{experiment}/{func}')
        path = path_in / f'freq{frequency}nn_w{nn_window}nn_p{3}'
        noNNReval= pd.read_csv(path/f'noNNReval_{measure_name[index_measure]}.csv').mean(0).values[0]

        #noNNRestart= pd.read_csv(path/'noNNRestart_mof.csv')

       # NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


       # NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
       # NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        #y=[noNNReval, NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        y=[noNNReval, NNnorm_Worst,NNdrop_Worst]

        total.append(y)
        #print(total)

    #,  index=[Func_names,Func_names]

#df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])
df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock', 'sphere','rastrigin','rosenbrock'])

#pd.options.display.float_format = '{:,.2f}'.format


print(pd.DataFrame.to_csv(df, float_format='%.2f'))


#pd.DataFrame.to_latex(df, multirow=True)    #, float_format="%%.2f" ,, , longtable=True,  bold_rows=True,


,noNNReval,NNnorm_Worst,NNdrop_Worst
sphere,20.86,5.88,10.26
rastrigin,31.97,31.36,30.68
rosenbrock,132878.68,95152.01,196130.84
sphere,0.49,0.23,0.28
rastrigin,2.63,3.09,2.84
rosenbrock,477.63,176.78,199.93
sphere,0.05,5.19,5.24
rastrigin,9.54,6.27,5.16
rosenbrock,106.15,3896.32,4206.74
sphere,114.51,78.15,90.07
rastrigin,180.10,90.70,94.25
rosenbrock,36238911.84,21532833.04,24626277.92



In [101]:
pd.read_csv('tmp.csv')

,experiment,function,NNdrop,NNnorm,noNNReval
0,exp1,rastrigin,30.68,31.36,31.97
1,exp1,rosenbrock,196130.84,95152.01,132878.68
2,exp1,sphere,10.26,5.88,20.86
3,exp2,rastrigin,2.84,3.09,2.63
4,exp2,rosenbrock,199.93,176.78,477.63
5,exp2,sphere,0.28,0.23,0.49
6,exp3,rastrigin,5.16,6.27,9.54
7,exp3,rosenbrock,4206.74,3896.32,106.15
8,exp3,sphere,5.24,5.19,0.05
9,exp4,rastrigin,94.25,90.70,180.10


In [53]:
df

,noNNReval,NNnorm_Worst,NNdrop_Worst
sphere,2.085900e+01,5.882522e+00,1.025944e+01
rastrigin,3.197001e+01,3.135821e+01,3.068299e+01
rosenbrock,1.328787e+05,9.515201e+04,1.961308e+05
sphere,4.901408e-01,2.253121e-01,2.824855e-01
rastrigin,2.626830e+00,3.090646e+00,2.835697e+00
rosenbrock,4.776291e+02,1.767826e+02,1.999253e+02
sphere,4.886981e-02,5.188318e+00,5.242722e+00
rastrigin,9.542240e+00,6.269999e+00,5.157696e+00
rosenbrock,1.061483e+02,3.896322e+03,4.206740e+03
sphere,1.145067e+02,7.815408e+01,9.006552e+01


In [ ]:
#report nn_time
measure_name=['mof','sr','arr','nfe','nn_time']
index_measure=4
total=[]
lables = [ 'NNnorm_Random', 'NNnorm_Worst','NNnorm_Closest','NNdrop_Random', 'NNdrop_Worst', 'NNdrop_Closest']

for experiment in exp_names:
    
    for func in Func_names:
        path = Path(f'../../data/results/{experiment}/{func}')

        NNnorm_Random= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Random = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]


        NNnorm_Worst= pd.read_csv(path/f'NNnorm_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Worst = pd.read_csv(path/f'NNdrop_Worst_{measure_name[index_measure]}.csv').mean(0).values[0]

        NNnorm_Closest= pd.read_csv(path/f'NNnorm_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        NNdrop_Closest = pd.read_csv(path/f'NNdrop_Closest_{measure_name[index_measure]}.csv').mean(0).values[0]
        
       # x=np.array([noNNReval.mof, NNnorm_Random.mof, NNnorm_Worst.mof, NNnorm_Closest.mof, NNdrop_Random.mof,NNdrop_Worst.mof,NNdrop_Closest.mof])
        y=[ NNnorm_Random,NNnorm_Worst,NNnorm_Closest,NNdrop_Random, NNdrop_Worst, NNdrop_Closest]
        total.append(y)
        #print(total)


df=pd.DataFrame(total, columns=lables,index=['sphere','rastrigin','rosenbrock','sphere','rastrigin','rosenbrock'])



print(pd.DataFrame.to_csv(df, float_format='%.2f'))
#df
